In [1]:
import numpy as np
np.set_printoptions(precision=3)

In [5]:
# number of elements and nodes
n_el = 3
n_node = 4
## connectivity: which nodes form each element
elements = np.array([[1,4],[2,4],[3,4]])
## node coordinates
nodes_X = np.array([[-1,1],[0,1],[1,1],[0,0]])
## parameters are stiffness for each element (arbitrary, just to show)
E = 10e11
A1 = 1e-2
A2 = 2e-2
A3 = 1e-2
A_vec = np.array([A1,A2,A3])
# array of stiffness matrices
Ke_array = []
for i in range(n_el):
    ## transformation matrix (get angle based on node coordinates)
    X1e = nodes_X[elements[i,0]-1,0]
    Y1e = nodes_X[elements[i,0]-1,1]
    X2e = nodes_X[elements[i,1]-1,0]
    Y2e = nodes_X[elements[i,1]-1,1]    
    le = np.sqrt( (X2e-X1e)**2+(Y2e-Y1e)**2)
    phi = np.arctan2(Y2e-Y1e,X2e-X1e)
    Re = np.array([[np.cos(phi),np.sin(phi),0,0],\
                   [-np.sin(phi),np.cos(phi),0,0],\
                   [0,0,np.cos(phi),np.sin(phi)],\
                   [0,0,-np.sin(phi),np.cos(phi)]])
    ## stiffness matrix in the local (rotated system aligned with the truss element)
    Ke_loc = (A_vec[i]*E/le)*np.array([[1,0,-1,0],\
                                       [0,0,0,0],\
                                       [-1,0,1,0],\
                                       [0,0,0,0]])
    ## transform Ke_loc to Ke
    Ke = np.dot(Re.transpose(),np.dot(Ke_loc,Re))
    print('Ke for element ',i)
    print(Ke)
    Ke_array.append(Ke)

# assemble global stiffness
K = np.zeros((n_node*2,n_node*2))
for i in range(n_el):
    # watching out for matlab vs python convention 
    ind1 = elements[i,0]-1 # node 1
    ind2 = elements[i,1]-1 # node 2
    # first row 
    K[ind1*2+0,ind1*2+0] += Ke_array[i][0,0]
    K[ind1*2+0,ind1*2+1] += Ke_array[i][0,1]
    K[ind1*2+0,ind2*2+0] += Ke_array[i][0,2]
    K[ind1*2+0,ind2*2+1] += Ke_array[i][0,3]
    # second row
    K[ind1*2+1,ind1*2+0] += Ke_array[i][1,0]
    K[ind1*2+1,ind1*2+1] += Ke_array[i][1,1]
    K[ind1*2+1,ind2*2+0] += Ke_array[i][1,2]
    K[ind1*2+1,ind2*2+1] += Ke_array[i][1,3]
    # third row
    K[ind2*2+0,ind1*2+0] += Ke_array[i][2,0]
    K[ind2*2+0,ind1*2+1] += Ke_array[i][2,1]
    K[ind2*2+0,ind2*2+0] += Ke_array[i][2,2]
    K[ind2*2+0,ind2*2+1] += Ke_array[i][2,3]
    # fourth row
    K[ind2*2+1,ind1*2+0] += Ke_array[i][3,0]
    K[ind2*2+1,ind1*2+1] += Ke_array[i][3,1]
    K[ind2*2+1,ind2*2+0] += Ke_array[i][3,2]
    K[ind2*2+1,ind2*2+1] += Ke_array[i][3,3]
print('K: global stiffness')
print(K)

('Ke for element ', 0)
[[ 3.536e+09 -3.536e+09 -3.536e+09  3.536e+09]
 [-3.536e+09  3.536e+09  3.536e+09 -3.536e+09]
 [-3.536e+09  3.536e+09  3.536e+09 -3.536e+09]
 [ 3.536e+09 -3.536e+09 -3.536e+09  3.536e+09]]
('Ke for element ', 1)
[[ 7.499e-23 -1.225e-06 -7.499e-23  1.225e-06]
 [-1.225e-06  2.000e+10  1.225e-06 -2.000e+10]
 [-7.499e-23  1.225e-06  7.499e-23 -1.225e-06]
 [ 1.225e-06 -2.000e+10 -1.225e-06  2.000e+10]]
('Ke for element ', 2)
[[ 3.536e+09  3.536e+09 -3.536e+09 -3.536e+09]
 [ 3.536e+09  3.536e+09 -3.536e+09 -3.536e+09]
 [-3.536e+09 -3.536e+09  3.536e+09  3.536e+09]
 [-3.536e+09 -3.536e+09  3.536e+09  3.536e+09]]
K: global stiffness
[[ 3.536e+09 -3.536e+09  0.000e+00  0.000e+00  0.000e+00  0.000e+00
  -3.536e+09  3.536e+09]
 [-3.536e+09  3.536e+09  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   3.536e+09 -3.536e+09]
 [ 0.000e+00  0.000e+00  7.499e-23 -1.225e-06  0.000e+00  0.000e+00
  -7.499e-23  1.225e-06]
 [ 0.000e+00  0.000e+00 -1.225e-06  2.000e+10  0.000e+00  0.000e+

In [6]:
## To solve the problem with matrix partitioning we need to be careful because the node 3 has free 'x' but constrained 'y'
## To do the partition we need to shift the corresponding rows and columns first and then partition 

#swap rows
aux = K[4,:] # storing the row corresponding to the x-equation of node 3
K[4,:] = K[5,:] # replacing the x-row of node 3 with the y-equation of node 3
K[5,:] = aux # and replace the y-equation of node 3 with the x-equation

#swap columns 
aux = K[:,4] # storing the column corresponding to the ux of node 3
K[:,4] = K[:,5] # replacing the ux node 3 with the uy of node 3
K[:,5] = aux # and replace the uy of node 3 with the ux of node 3

# solving the system with matrix partitioning, partion the K
n_E = 5
n_F = 3
K_E = K[0:n_E,0:n_E]
K_F = K[n_E:,n_E:]
K_EF = K[0:n_E,n_E:]
print('K_E')
print(K_E)
print('K_F')
print(K_F)
print('K_EF')
print(K_EF)

K_E
[[ 3.536e+09 -3.536e+09  0.000e+00  0.000e+00  0.000e+00]
 [-3.536e+09  3.536e+09  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  7.499e-23 -1.225e-06  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.225e-06  2.000e+10  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  3.536e+09]]
K_F
[[ 3.536e+09 -3.536e+09 -3.536e+09]
 [-3.536e+09  7.071e+09 -1.431e-06]
 [-3.536e+09 -1.431e-06  2.707e+10]]
K_EF
[[ 0.000e+00 -3.536e+09  3.536e+09]
 [ 0.000e+00  3.536e+09 -3.536e+09]
 [ 0.000e+00 -7.499e-23  1.225e-06]
 [ 0.000e+00  1.225e-06 -2.000e+10]
 [ 3.536e+09 -3.536e+09 -3.536e+09]]


In [11]:
# known displacements
d_E = np.zeros((n_E))
# known external forces
# remember now this corresponds to [node3x, node4x, node4y]
F_F = np.array([0,1e3,0])
# solving A\b
d_F = np.linalg.solve(K_F,F_F- np.dot(K_EF.transpose(),d_E))
print(d_F)

[3.828e-07 3.328e-07 5.000e-08]


In [10]:
# to get the stress we need strain of each bar 
l1 = np.sqrt(2)
l2 = 1
l3 = np.sqrt(2)
l1new = np.sqrt( (d_F[1]+1)**2 + (d_F[2]-1)**2)
l2new = np.sqrt( (d_F[1])**2 + (d_F[2]-1)**2)
l3new = np.sqrt( (d_F[1]-1-d_F[0])**2 + (d_F[2]-1)**2)
sigma1 = (l1new-l1)/l1*E
sigma2 = (l2new-l2)/l2*E
sigma3 = (l3new-l3)/l3*E
print(sigma1)
print(sigma2)
print(sigma3)

141421.3745329533
-49999.94462906443
0.0010990647210786426
